# Part 1: Preparing the Data

#### In this part of the project,  I will be validating the data and preparing it for analysis.

In [49]:
import numpy as np
import pandas as pd

In [50]:
# Import the .csv file.
df = pd.read_csv("violence_data.csv")

### Exploring the contours of the violence dataset.

In [51]:
df.head()

,RecordID,Country,Gender,Demographics Question,Demographics Response,Question,Survey Year,Value
0,1,Afghanistan,F,Marital status,Never married,... if she burns the food,01/01/2015,NaN
1,1,Afghanistan,F,Education,Higher,... if she burns the food,01/01/2015,10.1
2,1,Afghanistan,F,Education,Secondary,... if she burns the food,01/01/2015,13.7
3,1,Afghanistan,F,Education,Primary,... if she burns the food,01/01/2015,13.8
4,1,Afghanistan,F,Marital status,"Widowed, divorced, separated",... if she burns the food,01/01/2015,13.8


In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12600 entries, 0 to 12599
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   RecordID               12600 non-null  int64  
 1   Country                12600 non-null  object 
 2   Gender                 12600 non-null  object 
 3   Demographics Question  12600 non-null  object 
 4   Demographics Response  12600 non-null  object 
 5   Question               12600 non-null  object 
 6   Survey Year            12600 non-null  object 
 7   Value                  11187 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 787.6+ KB


In [53]:
# Determine the number of countries.
df['Country'].nunique()

70

In [54]:
# Look at types of demographics.
df['Demographics Question'].unique()

array(['Marital status', 'Education', 'Employment', 'Age', 'Residence'],
      dtype=object)

In [55]:
# Look at types of questions asked.
print("A husband is justified in hitting/beating his wife...")
df['Question'].unique()

A husband is justified in hitting/beating his wife...


array(['... if she burns the food',
       '... for at least one specific reason',
       '... if she argues with him',
       '... if she goes out without telling him',
       '... if she neglects the children',
       '... if she refuses to have sex with him'], dtype=object)

In [56]:
# Determine if each unique question is asked to respondents in each country.
df['RecordID'].nunique() / df['Question'].nunique()

70.0

In [57]:
# Determine if the number of male and female respondents in each country is the same.

male_greater_female = []
female_greater_male = []

for country in df['Country']:
    genders = df[df['Country'] == country]['Gender']

    num_males = sum(genders.str.count('M'))
    num_females = sum(genders.str.count('F'))

    if num_males < num_females:
        male_greater_female.append(country)
    elif num_males > num_females:
        female_greater_male.append(country)
        
print(male_greater_female)
print(female_greater_male)


[]
[]


### Determine the relationship between GDP per capita (a standard metric for a country's standard of living and agreement values in the violence dataset.

In [58]:
# Check the violence dataset to see when populations were surveyed.
df['Survey Year'].unique()

array(['01/01/2015', '01/01/2017', '01/01/2006', '01/01/2014',
       '01/01/2008', '01/01/2010', '01/01/2016', '01/01/2011',
       '01/01/2012', '01/01/2013', '01/01/2002', '01/01/2018',
       '01/01/2009', '01/01/2005', '01/01/2003', '01/01/2001',
       '01/01/2000', '01/01/2007'], dtype=object)

In [59]:
# Check if there were multiple survey dates for each country.
year_dict = {}

for country in df['Country']:
    year_dict[country] = df[df['Country'] == 'Afghanistan']['Survey Year'].nunique()

diff_years = []
for k in year_dict:
    if year_dict[k] > 1:
        diff_years.append(k)

print(diff_years)

[]


#### I now know that each country only has one unique survey date. Let's try to match each country to its GDP per capita, based on the year surveyed. 

In [202]:
# Import dataset from The World Bank (https://data.worldbank.org/indicator/NY.GDP.PCAP.CD)
gdp_df = pd.read_csv(r'/Users/MOON/Documents/data_projects/violence_data/API_NY.GDP.PCAP.CD_DS2_en_csv_v2_1307369/API_NY.GDP.PCAP.CD_DS2_en_csv_v2_1307369.csv')

In [203]:
gdp_df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,ABW,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,23512.602600,24985.993280,24713.698050,26189.435510,26647.938100,27980.880700,28281.350480,29007.693000,NaN,NaN
1,Afghanistan,AFG,GDP per capita (current US$),NY.GDP.PCAP.CD,59.773194,59.860874,58.458015,78.706388,82.095231,101.108305,...,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.228110,556.302002,524.162881,502.115487
2,Angola,AGO,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,3587.883798,4615.468028,5100.095808,5254.882338,5408.410496,4166.979684,3506.072885,4095.812942,3289.646664,2973.591160
3,Albania,ALB,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,4094.350334,4437.142885,4247.629984,4413.060861,4578.631994,3952.801215,4124.055726,4531.020806,5284.380184,5352.857411
4,Andorra,AND,GDP per capita (current US$),NY.GDP.PCAP.CD,NaN,NaN,NaN,NaN,NaN,NaN,...,40852.666780,43335.328860,38686.461260,39538.766720,41303.929370,35762.523070,37474.665410,38962.880350,41793.055260,40886.391160


In [204]:
# Clean data by getting rid of columns we don't need.
gdp_df = gdp_df.drop(['Country Code','Indicator Name', 'Indicator Code'], axis=1)
gdp_df.head()

,Country Name,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23512.602600,24985.993280,24713.698050,26189.435510,26647.938100,27980.880700,28281.350480,29007.693000,NaN,NaN
1,Afghanistan,59.773194,59.860874,58.458015,78.706388,82.095231,101.108305,137.594352,160.898589,129.108323,...,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.228110,556.302002,524.162881,502.115487
2,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3587.883798,4615.468028,5100.095808,5254.882338,5408.410496,4166.979684,3506.072885,4095.812942,3289.646664,2973.591160
3,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4094.350334,4437.142885,4247.629984,4413.060861,4578.631994,3952.801215,4124.055726,4531.020806,5284.380184,5352.857411
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,40852.666780,43335.328860,38686.461260,39538.766720,41303.929370,35762.523070,37474.665410,38962.880350,41793.055260,40886.391160


In [205]:
# To prepare to merge the two datasets, first convert survey dates in violence dataset to just years.
df['Survey Year'] = df['Survey Year'].apply(lambda d: d.split('/')[-1])
df['Survey Year'].head()

0    2015
1    2015
2    2015
3    2015
4    2015
Name: Survey Year, dtype: object

In [206]:
# Now merge the two datasets based on the year surveyed in the violence dataset. 
merged_df = df.merge(gdp_df, how='inner', left_on = 'Country', right_on = 'Country Name')
merged_df.head()

,RecordID,Country,Gender,Demographics Question,Demographics Response,Question,Survey Year,Value,Country Name,1960,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,1,Afghanistan,F,Marital status,Never married,... if she burns the food,2015,NaN,Afghanistan,59.773194,...,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487
1,1,Afghanistan,F,Education,Higher,... if she burns the food,2015,10.1,Afghanistan,59.773194,...,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487
2,1,Afghanistan,F,Education,Secondary,... if she burns the food,2015,13.7,Afghanistan,59.773194,...,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487
3,1,Afghanistan,F,Education,Primary,... if she burns the food,2015,13.8,Afghanistan,59.773194,...,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487
4,1,Afghanistan,F,Marital status,"Widowed, divorced, separated",... if she burns the food,2015,13.8,Afghanistan,59.773194,...,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487


In [207]:
# Clean the data: drop unnecessary columns and drop rows where 'Value' is NA (since this is the key metric we're analyzing here).
merged_df = merged_df.drop(['RecordID','Country Name'], axis=1)
merged_df = merged_df[merged_df['Value'].notna()]
merged_df.head()

,Country,Gender,Demographics Question,Demographics Response,Question,Survey Year,Value,1960,1961,1962,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
1,Afghanistan,F,Education,Higher,... if she burns the food,2015,10.1,59.773194,59.860874,58.458015,...,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487
2,Afghanistan,F,Education,Secondary,... if she burns the food,2015,13.7,59.773194,59.860874,58.458015,...,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487
3,Afghanistan,F,Education,Primary,... if she burns the food,2015,13.8,59.773194,59.860874,58.458015,...,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487
4,Afghanistan,F,Marital status,"Widowed, divorced, separated",... if she burns the food,2015,13.8,59.773194,59.860874,58.458015,...,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487
5,Afghanistan,F,Employment,Employed for kind,... if she burns the food,2015,17.0,59.773194,59.860874,58.458015,...,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487


In [208]:
# Some GDP values may be empty, so mark those as NA.
merged_df = merged_df.fillna(value=0)
merged_df.head()

,Country,Gender,Demographics Question,Demographics Response,Question,Survey Year,Value,1960,1961,1962,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
1,Afghanistan,F,Education,Higher,... if she burns the food,2015,10.1,59.773194,59.860874,58.458015,...,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487
2,Afghanistan,F,Education,Secondary,... if she burns the food,2015,13.7,59.773194,59.860874,58.458015,...,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487
3,Afghanistan,F,Education,Primary,... if she burns the food,2015,13.8,59.773194,59.860874,58.458015,...,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487
4,Afghanistan,F,Marital status,"Widowed, divorced, separated",... if she burns the food,2015,13.8,59.773194,59.860874,58.458015,...,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487
5,Afghanistan,F,Employment,Employed for kind,... if she burns the food,2015,17.0,59.773194,59.860874,58.458015,...,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487


In [209]:
# Since we dropped rows, we will need to reset the index.
merged_df = merged_df.reset_index(drop=True)
merged_df.head()

,Country,Gender,Demographics Question,Demographics Response,Question,Survey Year,Value,1960,1961,1962,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Afghanistan,F,Education,Higher,... if she burns the food,2015,10.1,59.773194,59.860874,58.458015,...,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487
1,Afghanistan,F,Education,Secondary,... if she burns the food,2015,13.7,59.773194,59.860874,58.458015,...,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487
2,Afghanistan,F,Education,Primary,... if she burns the food,2015,13.8,59.773194,59.860874,58.458015,...,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487
3,Afghanistan,F,Marital status,"Widowed, divorced, separated",... if she burns the food,2015,13.8,59.773194,59.860874,58.458015,...,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487
4,Afghanistan,F,Employment,Employed for kind,... if she burns the food,2015,17.0,59.773194,59.860874,58.458015,...,543.303042,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487


In [212]:
# Now create a new column with the country's GDP per capita based on its survey year.
merged_df['GDP per capita'] = None

for index, row in merged_df.iterrows():
    year = row['Survey Year']
    merged_df.iat[index,-1] = row[year]
    
merged_df.head()

,Country,Gender,Demographics Question,Demographics Response,Question,Survey Year,Value,1960,1961,1962,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,GDP per capita
0,Afghanistan,F,Education,Higher,... if she burns the food,2015,10.1,59.773194,59.860874,58.458015,...,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487,578.466
1,Afghanistan,F,Education,Secondary,... if she burns the food,2015,13.7,59.773194,59.860874,58.458015,...,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487,578.466
2,Afghanistan,F,Education,Primary,... if she burns the food,2015,13.8,59.773194,59.860874,58.458015,...,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487,578.466
3,Afghanistan,F,Marital status,"Widowed, divorced, separated",... if she burns the food,2015,13.8,59.773194,59.860874,58.458015,...,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487,578.466
4,Afghanistan,F,Employment,Employed for kind,... if she burns the food,2015,17.0,59.773194,59.860874,58.458015,...,591.162759,641.871479,637.165523,613.856689,578.466353,547.22811,556.302002,524.162881,502.115487,578.466


In [217]:
# Make sure that every row has a corresponding GDP per capita value.
merged_df['GDP per capita'].isnull().any()

False

In [218]:
# Since each country now has a GDP per capita assigned to it (based on survey year), get rid of unnecessary columns.
print(len(merged_df.columns))

idx = np.r_[7:67]
new_merged_df = merged_df.drop(merged_df.columns[idx], axis=1)
new_merged_df.head()

68


,Country,Gender,Demographics Question,Demographics Response,Question,Survey Year,Value,GDP per capita
0,Afghanistan,F,Education,Higher,... if she burns the food,2015,10.1,578.466
1,Afghanistan,F,Education,Secondary,... if she burns the food,2015,13.7,578.466
2,Afghanistan,F,Education,Primary,... if she burns the food,2015,13.8,578.466
3,Afghanistan,F,Marital status,"Widowed, divorced, separated",... if she burns the food,2015,13.8,578.466
4,Afghanistan,F,Employment,Employed for kind,... if she burns the food,2015,17.0,578.466


In [219]:
new_merged_df.to_csv('/Users/MOON/Documents/data_projects/violence_data/cleaned_violence_df.csv', index_label=None)

### Great! Now, let's move into a different jupyter notebook to begin analyzing.